In [89]:
'''this notebook heavily refers to the post https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/'''

In [128]:
import nltk
#nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#for Stemming
from nltk.stem.snowball import DanishStemmer

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [129]:
import os,sys
parentPath = os.path.join(os.path.dirname(os.getcwd()),"tools")
sys.path.insert(0, parentPath)
from scripts_py import preprocessing as pre

#### Prepare stopwords

In [130]:
# NLTK Stop words 
# TODO change to our own stop words list
from nltk.corpus import stopwords
stop_words = stopwords.words('danish')
stop_words.extend(['paa','ved','saa','den'])

#### read in articles

In [131]:
data = list()
#add any articles as you like
#articles = ["aare01val_workid69870.txt","aakjaer01val_workid55565.txt","aakjaer01val_workid55662.txt"
 #           ,"aakjaer01val_workid55881.txt"]
articles=os.listdir(os.path.join(os.path.dirname(os.getcwd()),"ADL","plain"))
for ele in articles:
    article = os.path.join(os.path.dirname(os.getcwd()),"ADL","plain",ele)
    with open(article, encoding ="utf-8") as f:
        lines = f.readlines()
    data.append( ' '.join(lines))

#### Preprocessing

In [132]:
nalpha_content = [pre.re_nalpha(ele) for ele in data]
tokenized_content = [pre.tokenizer(ele) for ele in nalpha_content]

In [ ]:
tokenized_content[:1]

In [134]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def stemmer(wordList):
    """stemming the words for later processing"""
    stemmer = DanishStemmer()
    return [stemmer.stem(tempWord) for tempWord in wordList]

In [135]:
data_words_nostops=remove_stopwords(tokenized_content)

In [136]:
bigram = gensim.models.Phrases(tokenized_content, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram = gensim.models.Phrases(bigram[tokenized_content], threshold=100)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

In [137]:

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_stemmed = [stemmer(ele) for ele in data_words_bigrams]
data_stemmed=remove_stopwords(data_stemmed)

In [138]:
# Create Dictionary
id2word = corpora.Dictionary(data_stemmed)
# Create Corpus
texts = data_stemmed

In [139]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [144]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [145]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.082*"så" + 0.019*"sen" + 0.010*"få" + 0.010*"fortvivl" + 0.009*"måt" + '
  '0.008*"arv" + 0.007*"gå" + 0.006*"ff" + 0.006*"flad" + 0.006*"skæg"'),
 (1,
  '0.039*"oss" + 0.029*"met" + 0.005*"aldri" + 0.004*"hemming" + 0.003*"wor" + '
  '0.003*"sua" + 0.003*"siin" + 0.002*"føst" + 0.002*"ac" + 0.002*"liff_oc"'),
 (2,
  '0.025*"dit" + 0.010*"hjert" + 0.009*"himl" + 0.009*"kun" + 0.008*"hver" + '
  '0.007*"sorg" + 0.007*"sød" + 0.007*"gud" + 0.007*"liv" + 0.006*"evig"'),
 (3,
  '0.086*"hr" + 0.049*"seng" + 0.027*"led" + 0.017*"slaa" + 0.013*"spilled" + '
  '0.013*"ked" + 0.012*"dø" + 0.012*"saamænd" + 0.011*"nar" + '
  '0.011*"kammerat"'),
 (4,
  '0.026*"hand" + 0.024*"ey" + 0.022*"gud" + 0.016*"kand" + 0.014*"naad" + '
  '0.011*"ieg" + 0.008*"udi" + 0.008*"vel" + 0.008*"mand" + 0.008*"dend"'),
 (5,
  '0.055*"gansk" + 0.039*"vist" + 0.016*"grund" + 0.016*"tre" + 0.015*"frøk" + '
  '0.014*"aa" + 0.012*"mørk" + 0.011*"mindr" + 0.010*"fruen" + 0.009*"naa"'),
 (6,
  '0.026*"saae" + 0

In [126]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_stemmed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.63565437474

Coherence Score:  0.436928849656


In [146]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      28.245084        1       1  0.397766  0.018829
18     23.555603        1       2  0.353538  0.051559
17     10.168688        1       3  0.240146 -0.243971
19      8.923946        1       4  0.283259  0.074041
6       3.891560        1       5 -0.076565  0.093430
4       3.151664        1       6  0.057407  0.095418
9       2.674081        1       7 -0.013003 -0.172662
10      2.356782        1       8 -0.066169  0.004652
2       2.267566        1       9  0.020203  0.207355
7       1.971352        1      10 -0.118962  0.011571
0       1.861254        1      11 -0.103381 -0.069379
5       1.709761        1      12 -0.037154 -0.180455
11      1.619103        1      13 -0.043358  0.124110
16      1.505237        1      14 -0.127437  0.123884
12      1.497456        1      15 -0.075588 -0.118194
13      1.293371        1      16 -0.125027  0.051572
14      1.166066        1      17 -0.105943  0.062243
1       0.825993        1      18 -0.159165 -0.032167
15      0.710153        1      19 -0.156196 -0.033917
3       0.605281        1      20 -0.144374 -0.067919, topic_info=       Category          Freq         Term         Total  loglift  logprob
term                                                                      
7212    Default  68399.000000         sagd  68399.000000  30.0000  30.0000
140     Default  80396.000000          kom  80396.000000  29.0000  29.0000
36427   Default  21730.000000           ei  21730.000000  28.0000  28.0000
20683   Default  19801.000000           så  19801.000000  27.0000  27.0000
84745   Default  18699.000000           oc  18699.000000  26.0000  26.0000
631     Default  30434.000000          gud  30434.000000  25.0000  25.0000
1674    Default  20121.000000         kong  20121.000000  24.0000  24.0000
1100    Default  49337.000000           ja  49337.000000  23.0000  23.0000
3034    Default  12187.000000        gansk  12187.000000  22.0000  22.0000
1683    Default  44200.000000          kun  44200.000000  21.0000  21.0000
8633    Default  12855.000000          ans  12855.000000  20.0000  20.0000
807     Default  21978.000000           se  21978.000000  19.0000  19.0000
2333    Default  33399.000000          liv  33399.000000  18.0000  18.0000
5256    Default  13615.000000         vist  13615.000000  17.0000  17.0000
706     Default  44616.000000         kund  44616.000000  16.0000  16.0000
36566   Default  12968.000000         saae  12968.000000  15.0000  15.0000
136     Default  70670.000000          kan  70670.000000  14.0000  14.0000
7276    Default  17760.000000        hjert  17760.000000  13.0000  13.0000
101     Default  32950.000000          vel  32950.000000  12.0000  12.0000
10      Default  13594.000000          dit  13594.000000  11.0000  11.0000
4214    Default  10646.000000         hand  10646.000000  10.0000  10.0000
1164    Default  33468.000000          maa  33468.000000   9.0000   9.0000
21883   Default  15515.000000          gør  15515.000000   8.0000   8.0000
212     Default  35907.000000          gik  35907.000000   7.0000   7.0000
364     Default  30506.000000         mand  30506.000000   6.0000   6.0000
4627    Default   6754.000000           hr   6754.000000   5.0000   5.0000
144     Default  27395.000000          lad  27395.000000   4.0000   4.0000
79      Default  34626.000000        skuld  34626.000000   3.0000   3.0000
3063    Default  13873.000000          att  13873.000000   2.0000   2.0000
32993   Default   9760.000000           ey   9760.000000   1.0000   1.0000
...         ...           ...          ...           ...      ...      ...
3894    Topic20     53.514932        digat     54.654422   5.0862  -7.2923
522274  Topic20     47.937402     fensmark     49.076927   5.0837  -7.4023
2340    Topic20     47.372035          ral     48.511525   5.0835  -7.4142
4497    Topic20     45.985602        alkov     47.125091   5.0828  -7